# BE - Semantic Segmentation
## A Salt Identification Case Study

# Import

In [ ]:
import  numpy as np
import cv2
from matplotlib import pyplot as plt
from scipy import ndimage
import keras


# Sommaire

[I. Loading and exploring data](#Partie1) Inference

[II. UNET Architecture and Training](#Partie2)
- [II.1. Architecture definition](#Partie2.1)
- [II.1. Training](#Partie2.2)

[III. Inference](#Partie3)
- [III.1. Training set](#Partie3.1)
- [III.1. testing set](#Partie3.2)

[IV. Questions](#Partie4)
- [IV.1. Max pooling](#Partie4.1)
- [IV.2. Skip Connections](#Partie4.2)
- [IV.3. FCN and Auto-encoder](#Partie4.3)
- [IV.4. Threshold for inference ](#Partie4.4)
